# Agentes com Autogen - Function Calling - Hands-On

Agradecimento ao professor Julio Cesar dos Reis pelo material.

## Descrição

Exploramos a implementação de agentes simples utilizando a biblioteca Autogen. O objetivo é demonstrar como configurar e interagir com agentes capazes de executar tarefas automatizadas de forma eficiente. Aprenderemos a estruturar prompts, configurar agentes com diferentes especializações e testar interações dinâmicas entre eles.


In [4]:
!pip install autogen
!pip install --upgrade pydantic
!pip install ipywidgets
!pip install groq

In [5]:
from typing import Literal
import autogen
from typing_extensions import Annotated
import getpass

In [6]:
# Solicitar a API key de forma segura
from google.colab import userdata
api_key =  userdata.get('groq')

config_list = [
    {
      "model": "llama3.3-70b-versatile",
      "api_key": api_key,
      "api_type": "groq"
    }
]

SecretNotFoundError: Secret groq does not exist.

Uma ferramenta deve ser registrada com pelo menos dois agentes para que seja útil na conversação. O agente registrado com a assinatura da ferramenta por meio de register_for_llm pode chamar a ferramenta; o agente registrado com o objeto de função da ferramenta por meio de register_for_execution pode executar a função da ferramenta.

In [ ]:
llm_config = {
    "config_list": config_list,
    "timeout": 120
}

#agente que efetua conversão de moeda
currency_bot = autogen.AssistantAgent(
    name="currency_bot",
    system_message="Para tarefas de troca de moeda, use apenas as funções que você recebeu. Responda TERMINATE"
                   "quando a tarefa estiver concluída.",
    llm_config=llm_config
)

#agente user proxy
user_proxy = autogen.UserProxyAgent(
    name="user_proxy",
    is_termination_msg=lambda x: x.get("content", "") and x.get("content", "").rstrip().endswith("TERMINATE"),
    human_input_mode="NEVER",
    max_consecutive_auto_reply=1,
    code_execution_config=False
)

CurrencySymbol = Literal["USD", "EUR"]

#função de taxa_de_câmbio
def exchange_rate(base_currency: CurrencySymbol, quote_currency: CurrencySymbol) -> float:
    if base_currency == quote_currency:
        return 1.0
    elif base_currency == "USD" and quote_currency == "EUR":
        return 1 / 1.09
    elif base_currency == "EUR" and quote_currency == "USD":
        return 1 / 1.1
    else:
        raise ValueError(f"Moedas desconhecidas: {base_currency}, {quote_currency}")  # Tradução: "Moedas desconhecidas:..."

#registrando a ferramenta
@user_proxy.register_for_execution()
# Register the tool signature with the assistant agent.
@currency_bot.register_for_llm(description="Calculadora de câmbio de moeda")  # Tradução: "Calculadora de câmbio de moeda"
def currency_calculator(
        base_amount: Annotated[float, "Quantia da moeda em base_currency"],  # Tradução: "Quantia da moeda em base_currency"
        base_currency: Annotated[CurrencySymbol, "Moeda base"] = "USD",  # Tradução: "Moeda base"
        quote_currency: Annotated[CurrencySymbol, "Moeda de cotação"] = "EUR"  # Tradução: "Moeda de cotação"
) -> str:
    quote_amount = exchange_rate(base_currency, quote_currency) * base_amount
    return f"{quote_amount} - {quote_currency}"



#######################################
#inicialização da conversa
user_proxy.initiate_chat(
    currency_bot,
    message="E ai, pai? Tenho 200 dolares e estou indo pra europa, quanto eu tenho na moeda local?"  # Tradução: "Converter 100 USD para EUR"
    # message="E ai, truta! Está calor, hein? Estou na sala"

)


# Seu código vem aqui abaixo
$\color{red}{\text{Agora é sua vez, cria um agente com seu objetivo e suas ações}}$

In [ ]:
llm_config = {
    "config_list": config_list,
    "timeout": 120
}

#agente que efetua conversão de moeda
currency_bot = autogen.AssistantAgent(
    name="currency_bot",
    system_message="Para tarefas relacionadas com agendamento de reuniões, use apenas as funções que você recebeu. Responda TERMINATE "
                   "quando a tarefa estiver concluída.",
    llm_config=llm_config
)

#agente user proxy
user_proxy = autogen.UserProxyAgent(
    name="user_proxy",
    is_termination_msg=lambda x: x.get("content", "") and x.get("content", "").rstrip().endswith("TERMINATE"),
    human_input_mode="NEVER",
    max_consecutive_auto_reply=1,
    code_execution_config=False
)

EnviromentSymbol = Literal["escritório", "garagem", "jardim"]

# Função para tocar música em um ambiente
#registrando a ferramenta
@user_proxy.register_for_execution()
# Register the tool signature with the assistant agent.
@currency_bot.register_for_llm(description="Toca uma música em algum ambiente")
def def play_music(
    ambiente: Annotated[EnviromentSymbol, "Ambiente onde a música será tocada"],
    musica: Annotated[str, "Nome da música para tocar"]
) -> str:
    return f"Tocando '{musica}' no {ambiente}"

# Função para ajustar o volume da música
@user_proxy.register_for_execution()
@music_bot.register_for_llm(description="Ajusta o volume da música em algum ambiente")
def set_volume(
    ambiente: Annotated[EnviromentSymbol, "Ambiente onde o volume será ajustado"],
    volume: Annotated[int, "Volume desejado, de 0 a 100"]
) -> str:
    return f"Volume ajustado para {volume}% no {ambiente}"


#######################################
#inicialização da conversa
user_proxy.initiate_chat(
    music_bot,
    message="E ai, truta! Está animado? Quero tocar uma música no escritório"
)


